# Model Comparison

This notebook compares multiple machine learning models for predicting NBA All-Star selections using season-level player and team data. 

We train baseline Random Forest and Logistic Regression models, then tune their hyperparameters using randomized search. Model performance is evaluated on the test set using classification metrics to identify the best-performing and most interpretable model for the task.

The goal is to select a final model that balances predictive accuracy with explainability to support future analysis and deployment.

In [1]:
import config
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

In [2]:
# load preprocessed data
X = pd.read_pickle("../data/clean/X_data.pkl")
y = pd.read_pickle("../data/clean/y_data.pkl")

In [3]:
# split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=config.test_size,
    stratify=y if config.stratify else None,
    random_state=config.random_state
)

# save test data for later use in model_evaluation notebook
X_test.to_pickle("../data/clean/X_test.pkl")
y_test.to_pickle("../data/clean/y_test.pkl")

## Model Training and Hyperparameter Tuning

In this section, we train baseline and tuned versions of both the Random Forest and Logistic Regression classifiers.

First, we train baseline models using default or initial hyperparameters as defined in the configuration. These baseline models provide a performance benchmark.

Next, we apply randomized hyperparameter search (`RandomizedSearchCV`) to each model to find better hyperparameter settings that optimize the F1 score on validation folds.

### Random Forest

In [4]:
# train random forest model
rf_model = RandomForestClassifier(**config.rf_params)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_features=None, random_state=66)

In [5]:
# use trained model to predict test and evaluate
rf_preds = rf_model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_preds))
print("Random Forest - Classification Report:")
print(classification_report(y_test, rf_preds))

Confusion Matrix:
 [[3771   40]
 [  84  156]]
Random Forest - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3811
           1       0.80      0.65      0.72       240

    accuracy                           0.97      4051
   macro avg       0.89      0.82      0.85      4051
weighted avg       0.97      0.97      0.97      4051



In [6]:
# hyperparameter tuning
rf_search_model = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=config.random_state),
    param_distributions=config.rf_param_dist,
    **config.search_cv_params
)
rf_search_model.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=66),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=66, scoring='f1', verbose=1)

In [7]:
# evaluate
rf_search_preds = rf_search_model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_search_preds))
print("Random Forest (Tuned) - Classification Report:")
print(classification_report(y_test, rf_search_preds))

Confusion Matrix:
 [[3742   69]
 [  79  161]]
Random Forest (Tuned) - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3811
           1       0.70      0.67      0.69       240

    accuracy                           0.96      4051
   macro avg       0.84      0.83      0.83      4051
weighted avg       0.96      0.96      0.96      4051



### Logistic Regression

In [8]:
# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# save scaler for later use in model_evaluation notebook (if final chosen model is an LR model)
with open("../models/logreg_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [9]:
# train logistic regression model
logreg_model = LogisticRegression(**config.logreg_params)
logreg_model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=66)

In [10]:
# evaluate
logreg_preds = logreg_model.predict(X_test_scaled)
print("Confusion Matrix:\n", confusion_matrix(y_test, logreg_preds))
print("Logistic Regression - Classification Report:")
print(classification_report(y_test, logreg_preds))

Confusion Matrix:
 [[3765   46]
 [  81  159]]
Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3811
           1       0.78      0.66      0.71       240

    accuracy                           0.97      4051
   macro avg       0.88      0.83      0.85      4051
weighted avg       0.97      0.97      0.97      4051



In [11]:
# hyperparameter tuning
logreg_search_model = RandomizedSearchCV(
    estimator=LogisticRegression(),
    param_distributions=config.logreg_param_dist,
    **config.search_cv_params
)
logreg_search_model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/Users/paulfox/opt/anaconda3/envs/eda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 14 is smaller than n_iter=20. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=20, n_jobs=-1,
                   param_distributions={'C': [0.01, 0.1, 1, 10, 100, 1000,
                                              10000],
                                        'class_weight': [None, 'balanced'],
                                        'penalty': ['l2']},
                   random_state=66, scoring='f1', verbose=1)

In [12]:
# evaluate
logreg_search_preds = logreg_search_model.predict(X_test_scaled)
print("Confusion Matrix:\n", confusion_matrix(y_test, logreg_search_preds))
print("Logistic Regression (Tuned) - Classification Report:")
print(classification_report(y_test, logreg_search_preds))

Confusion Matrix:
 [[3765   46]
 [  81  159]]
Logistic Regression (Tuned) - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3811
           1       0.78      0.66      0.71       240

    accuracy                           0.97      4051
   macro avg       0.88      0.83      0.85      4051
weighted avg       0.97      0.97      0.97      4051



## Model Comparison

Now that we have our trained baseline and tuned Random Forest and Logistic Regression models, we'll compare their performance using key classification metrics.

This comparison helps identify which model offers the best balance of F1 score, precision, recall, and accuracy for predicting NBA All-Star selections. The selected model will be saved for further analysis and future use.

In [13]:
# f1, precision, and recall scores for all models
rf_f1 = f1_score(y_test, rf_preds)
rf_search_f1 = f1_score(y_test, rf_search_preds)
logreg_f1 = f1_score(y_test, logreg_preds)
logreg_search_f1 = f1_score(y_test, logreg_search_preds)

rf_precision = precision_score(y_test, rf_preds)
rf_search_precision = precision_score(y_test, rf_search_preds)
logreg_precision = precision_score(y_test, logreg_preds)
logreg_search_precision = precision_score(y_test, logreg_search_preds)

rf_recall = recall_score(y_test, rf_preds)
rf_search_recall = recall_score(y_test, rf_search_preds)
logreg_recall = recall_score(y_test, logreg_preds)
logreg_search_recall = recall_score(y_test, logreg_search_preds)

print(f"Random Forest - F1: {rf_f1:.3f}, Precision: {rf_precision:.3f}, Recall: {rf_recall:.3f}")
print(f"Tuned Random Forest - F1: {rf_search_f1:.3f}, Precision: {rf_search_precision:.3f}, Recall: {rf_search_recall:.3f}")
print(f"Logistic Regression - F1: {logreg_f1:.3f}, Precision: {logreg_precision:.3f}, Recall: {logreg_recall:.3f}")
print(f"Tuned Logistic Regression - F1: {logreg_search_f1:.3f}, Precision: {logreg_search_precision:.3f}, Recall: {logreg_search_recall:.3f}")


Random Forest - F1: 0.716, Precision: 0.796, Recall: 0.650
Tuned Random Forest - F1: 0.685, Precision: 0.700, Recall: 0.671
Logistic Regression - F1: 0.715, Precision: 0.776, Recall: 0.662
Tuned Logistic Regression - F1: 0.715, Precision: 0.776, Recall: 0.662


## Final Model Selection

Although the base Random Forest model achieved the highest F1 score by a narrow margin (0.716 vs. 0.715), I’ve chosen the base Logistic Regression model as the final model for this project.

F1 score was established as the primary success metric, and both models performed nearly identically on that front. However, the Logistic Regression model delivered slightly better recall (0.662 vs. 0.650), which aligns more closely with the project’s goal of correctly identifying as many actual All-Stars as possible. It also offers greater interpretability and simplicity, making it easier to understand and explain the factors influencing predictions.

Given the nearly identical F1 score, stronger recall, and improved transparency, the base Logistic Regression model is the best overall choice and will be used as the final model moving forward.

In [14]:
# save the best model
with open("../models/all_star_model.pkl", "wb") as f:
    pickle.dump(logreg_model, f)